# Statistical Inference: Crime Rates in NYC

## Import packages and data

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm
df = pd.read_csv('data/final_NYC_crimes.csv')

## Question 1: NYC Census Population vs. NYC Crime Victim Population

**Null Hypothesis**: The sample mean between the population mean of NYC in 2019 by race **is the same** as the population mean of crime victims by race in NYC 2019

**Alternate Hypothesis**: The sample mean between the population mean of NYC in 2019 by race **is different** from the population mean of crime victims by race in NYC 2019

In [2]:
two_way_table = pd.crosstab(index=df["VIC_RACE"], columns=df["OFFENSE_NAME"])
two_way_table

OFFENSE_NAME,ARSON,ASSAULT,BURGLARY,DRIVING_UNDER_INFLUENCE,FORGERY,FRAUD,GAMBLING,HARRASSMENT,HOMICIDE,KIDNAPPING,...,MURDER,OFFENSES_AGAINST_PUBLIC_ORDER,POSSESSION_CONTROLLED_SUBSSTANCE,POSSESSION_WEAPON,ROBBERY,SEX_CRIMES,SOCIAL_RELATED_CRIMES,THEFT,TRAFFIC_LAWS_VIOLATION,UNCLASSIFIED
VIC_RACE,,,,,,,,,,,,,,,,,,,,,
AMERICAN INDIAN/ALASKAN NATIVE,0,433,45,2,12,23,0,560,57,0,...,1,387,3,7,118,29,9,47,49,0
ASIAN / PACIFIC ISLANDER,10,5906,1149,45,56,470,1,5599,773,5,...,20,4685,12,82,1712,594,67,373,826,4
BLACK,37,28116,1504,87,138,932,0,26398,4225,38,...,164,18736,52,441,3386,2437,444,327,2348,17
BLACK HISPANIC,9,4820,304,12,27,141,0,3772,502,8,...,19,2974,11,50,928,429,63,50,355,4
UNKNOWN,537,4797,4526,4052,4205,2165,288,4901,4008,12,...,4,27723,13209,5434,2018,642,304,1857,1750,840
WHITE,31,9333,2212,83,142,1748,0,13498,1958,17,...,14,10326,42,97,1621,1331,163,220,1321,23
WHITE HISPANIC,51,19697,1207,85,103,560,0,16426,2536,46,...,70,12316,38,169,3542,2073,217,179,1602,11


### Chi-square Goodness-of-fit Test

*Following http://hamelg.blogspot.com/2015/11/python-for-data-analysis-part-25-chi.html*

The one-way t-test checks whether a sample mean differs from the an expected (population) mean. The chi-squared goodness-of-fit test is an analog of the one-way t-test for categorical variables: it tests whether the distribution of sample categorical data matches an expected distribution.

**We will use the chi-square goodness-of-fit test to test our Null Hypothesis.**

In [3]:
# Get the count of victim race in the sample
df.VIC_RACE.value_counts()

UNKNOWN                           133583
BLACK                             113960
WHITE HISPANIC                     77877
WHITE                              70429
ASIAN / PACIFIC ISLANDER           33968
BLACK HISPANIC                     18526
AMERICAN INDIAN/ALASKAN NATIVE      2632
Name: VIC_RACE, dtype: int64

In [4]:
# Using population count from outside source
nyc_pop = pd.DataFrame(["white"]*2737163 + ["hispanic"]*2489089 +\
                        ["black"]*1899379 + ["asian"]*1247479 + ["other"]*164563)
           
# Use population counts from df.VIC_RACE.value_counts()
vic_pop = pd.DataFrame(["white"]*70429 + ["hispanic"]*148306 +\
                        ["black"]*113960 + ["asian"]*33968 + ["other"]*2632)

# Create crosstab of sample and population data
nyc_table = pd.crosstab(index=nyc_pop[0], columns="count")
vic_table = pd.crosstab(index=vic_pop[0], columns="count")

print( "NYC Population 2019")
print(nyc_table)
print(" ")
print( "Victim Population")
print(vic_table)

NYC Population 2019
col_0       count
0                
asian     1247479
black     1899379
hispanic  2489089
other      164563
white     2737163
 
Victim Population
col_0      count
0               
asian      33968
black     113960
hispanic  148306
other       2632
white      70429


Chi-squared tests are based on the so-called chi-squared statistic. You calculate the chi-squared statistic with the following formula:

*sum(((observed−expected)** 2) / expected)*

In the formula, observed is the actual observed count for each category and expected is the expected count based on the distribution of the population for the corresponding category. 

**Let's calculate the chi-squared statistic for our data.

In [5]:
# observed is the sample
observed = vic_table

# nyc_ratios has the population ratio for each race
nyc_ratios = nyc_table/len(nyc_pop)  

# expected is the product of the population ratios and the sample size
expected = nyc_ratios * len(vic_pop)   # Get expected counts

chi_squared_stat = (((observed-expected)**2)/expected).sum()

print(chi_squared_stat)

col_0
count    57318.734242
dtype: float64


From this, we can conclude that **the calculated chi-square statistic is 57318.734242.**

*Note: The chi-squared test assumes none of the expected counts are less than 5.

Similar to the t-test where we compared the t-test statistic to a critical value based on the t-distribution to determine whether the result is significant, in the chi-square test we compare the chi-square test statistic to a critical value based on the chi-square distribution. 

The scipy library shorthand for the chi-square distribution is **chi2**. 

Let's use this knowledge to find the critical value for 95% confidence level and check the p-value of our result:

In [6]:
# crit is the critical value (one-sided test ?)
crit = stats.chi2.ppf(q = 0.95, # Find the critical value for 95% confidence*
                      df = 1)   # Df = number of variable categories - 1


# p_value is the probability of obtaining results as extreme as the observed results of a statistical hypothesis test)
p_value = 1 - stats.chi2.cdf(x=chi_squared_stat,  # Find the p-value
                             df=1)

print("Critical value:", crit)
print("P value:", p_value)

Critical value: 3.841458820694124
P value: [0.]


From this, we conclude since our chi-squared statistic exceeds the critical value, we'd reject the null hypothesis that the two distributions are the same.

You can also carry out a chi-squared goodness-of-fit test automatically using the scipy function `scipy.stats.chisquare()`:

In [7]:
# You can carry out a chi-squared goodness-of-fit test automatically using the scipy function
stats.chisquare(f_obs= observed,   # Array of observed counts
                f_exp= expected)   # Array of expected counts

Power_divergenceResult(statistic=array([57318.7342417]), pvalue=array([0.]))

**Since our chi-squared statistic exceeds the critical value, we'd reject the null hypothesis that the two distributions are the same.**

## Question 2: Victim Race Population Independency

### Chi-square Independence Test
Independence is a key concept in probability that describes a situation where knowing the value of one variable tells you nothing about the value of another.

**We will use the chi-square independence test to test whether:**

H0: The victim race population is related to the crimes reported in NYC.

Ha: The victim race population is not related to the crimes reported in NYC.

In [8]:
# Count the ratio of each races relative to the whole crime report

# remove rows with race "UNKNOWN"
new_df = df[df.VIC_RACE != "UNKNOWN"]
# new_df = df


# Get frequency of each race
count_race = new_df.VIC_RACE.value_counts()

races = count_race.index

# list of objects used to construct the df
d = []

for race in races:
    count = count_race[race]
    ratio = count_race[race] / len(new_df.VIC_RACE)
    d.append(
        {
            'RACE': race,
            'COUNT': count,
            'RATIO': np.round(ratio, 4),
        }
    )
            
# Create the df using the list
df_race = pd.DataFrame(d)
df_race

,RACE,COUNT,RATIO
0,BLACK,113960,0.3590
1,WHITE HISPANIC,77877,0.2454
2,WHITE,70429,0.2219
3,ASIAN / PACIFIC ISLANDER,33968,0.1070
4,BLACK HISPANIC,18526,0.0584
5,AMERICAN INDIAN/ALASKAN NATIVE,2632,0.0083


In [9]:
# Count the ratio of each crimes relative to the whole crime report
count_offense = new_df.OFFENSE_NAME.value_counts()

all_offenses = count_offense.index

# list of objects used to construct the df
d = []

for offense in all_offenses:
    count = count_offense[offense]
    ratio = count_offense[offense] / len(new_df.OFFENSE_NAME)
    # Append object to list
    d.append(
        {
            'OFFENSE': offense,
            'COUNT': count,
            'RATIO': np.round(ratio, 4),
        }
    )
    
# Create the df using the list
df_offense = pd.DataFrame(d)
df_offense

,OFFENSE,COUNT,RATIO
0,LARCENY,83811,0.2641
1,ASSAULT,68305,0.2152
2,HARRASSMENT,66253,0.2087
3,OFFENSES_AGAINST_PUBLIC_ORDER,49424,0.1557
4,ROBBERY,11307,0.0356
5,HOMICIDE,10051,0.0317
6,SEX_CRIMES,6893,0.0217
7,TRAFFIC_LAWS_VIOLATION,6501,0.0205
8,BURGLARY,6421,0.0202
9,FRAUD,3874,0.0122


In [10]:
np.random.seed(11)

# Sample data randomly at fixed probabilities
# Create a random sample distribution of victim race
victim_race = np.random.choice(a= df_race.RACE,
                              p = df_race.RATIO,
                              size=10000)

# Sample data randomly at fixed probabilities
# Create a random sample distribution of offense type
offense = np.random.choice(a=df_offense.OFFENSE,
                              p = df_offense.RATIO,
                              size=10000)

victims = pd.DataFrame({"victim_race":victim_race, 
                       "offense":offense})

# Create the crosstab of random dist
victim_tab = pd.crosstab(victims.offense, victims.victim_race, margins = True)

#  # Get table without totals for later use
observed = victim_tab.iloc[0:5,0:3]  

victim_tab

victim_race,AMERICAN INDIAN/ALASKAN NATIVE,ASIAN / PACIFIC ISLANDER,BLACK,BLACK HISPANIC,WHITE,WHITE HISPANIC,All
offense,,,,,,,
ARSON,1,0,1,0,0,1,3
ASSAULT,18,229,757,106,478,530,2118
BURGLARY,0,17,77,16,37,59,206
DRIVING_UNDER_INFLUENCE,0,1,4,0,1,2,8
FORGERY,0,1,2,1,2,4,10
FRAUD,1,13,34,9,33,41,131
HARRASSMENT,15,197,755,122,444,470,2003
HOMICIDE,2,35,124,17,68,91,337
KIDNAPPING,0,1,3,0,1,0,5


**For a test of independence, we use the same chi-squared formula that we used for the goodness-of-fit test.** 


The main difference is we have to calculate the expected counts of each cell in a 2-dimensional table instead of a 1-dimensional table. To get the expected count for a cell, multiply the row total for that cell by the column total for that cell and then divide by the total number of observations. We can quickly get the expected counts for all cells in the table by taking the row totals and column totals of the table, performing an outer product on them with the np.outer() function and dividing by the number of observations:

In [11]:
# Calculate the expected distribution using the crosstab of randomly sampled data
expected =  np.outer(victim_tab["All"],
                     victim_tab.loc["All"]) / 10000

expected = pd.DataFrame(expected)
expected.columns = df_race.RACE.tolist() + ["All"]
# Pluck out Gambling (0 occurences) from the dataframe then reselect the OFFENSE column. Convert to list and add ["All"]
expected.index = df_offense[df_offense.OFFENSE != "GAMBLING"].OFFENSE.tolist() + ["All"]
expected

,BLACK,WHITE HISPANIC,WHITE,ASIAN / PACIFIC ISLANDER,BLACK HISPANIC,AMERICAN INDIAN/ALASKAN NATIVE,All
LARCENY,0.0219,0.3108,1.086,0.1749,0.6732,0.7332,3.0
ASSAULT,15.4614,219.4248,766.716,123.4794,475.2792,517.6392,2118.0
HARRASSMENT,1.5038,21.3416,74.572,12.0098,46.2264,50.3464,206.0
OFFENSES_AGAINST_PUBLIC_ORDER,0.0584,0.8288,2.896,0.4664,1.7952,1.9552,8.0
ROBBERY,0.0730,1.0360,3.620,0.5830,2.2440,2.4440,10.0
HOMICIDE,0.9563,13.5716,47.422,7.6373,29.3964,32.0164,131.0
SEX_CRIMES,14.6219,207.5108,725.086,116.7749,449.4732,489.5332,2003.0
TRAFFIC_LAWS_VIOLATION,2.4601,34.9132,121.994,19.6471,75.6228,82.3628,337.0
BURGLARY,0.0365,0.5180,1.810,0.2915,1.1220,1.2220,5.0
FRAUD,19.7392,280.1344,978.848,157.6432,606.7776,660.8576,2704.0


Now we can follow the same steps we took before to calculate the chi-square statistic, the critical value and the p-value:

*Note: We call .sum() twice: once to get the column sums and a second time to add the column sums together, returning the sum of the entire 2D table.*

In [12]:
# Compute the chi-squared stat
chi_squared_stat = (((observed-expected)**2)/expected).sum().sum()
print(chi_squared_stat)

199528.30020088967


**The computed chi-squared statistic is 199528.30020088967**

We will now compute the critical value and p-value:

In [13]:
# Compute a critical value 
crit = stats.chi2.ppf(q = 0.95, # Find the critical value for 95% confidence*
                      df = 24)   # *


# Compute a p-value
p_value = 1 - stats.chi2.cdf(x=chi_squared_stat,  # Find the p-value
                             df=8)

print("Critical value:", crit)
print("P value:", np.round(p_value, 3))

Critical value: 36.41502850180731
P value: 0.0


From this, we can see that **the critical value is  36.41502850180731** and **the p-value is 0.0**

The chi-squared statistics is **199528.30020088967** which is way higher than our critical value of **36.42**.

(Source: https://www.stat.purdue.edu/~lfindsen/stat503/Chi-Square.pdf)


Since the p-value is 0, and having a way higher chi-squared statistics, we can reject the null hypothesis.

As with the goodness-of-fit test, we can use scipy to conduct a test of independence quickly. Use `stats.chi2_contingency()` function to conduct a test of independence automatically given a frequency table of observed counts:

In [14]:
stats.chi2_contingency(observed = observed)

(31.157932571320597,
 0.0001316936897183417,
 8,
 array([[3.42960289e-02, 4.47653430e-01, 1.51805054e+00],
        [1.72166065e+01, 2.24722022e+02, 7.62061372e+02],
        [1.61191336e+00, 2.10397112e+01, 7.13483755e+01],
        [8.57400722e-02, 1.11913357e+00, 3.79512635e+00],
        [5.14440433e-02, 6.71480144e-01, 2.27707581e+00]]))

 The output shows the chi-square statistic, the p-value and the degrees of freedom followed by the expected counts.
As expected, given the high p-value, the test result **does not detect a significant relationship between the variables**.

## Experimenting

Trying to see if this statitistics is good enough.

In [15]:
# Create a contingency table 
stats.chi2_contingency(observed= observed)

(31.157932571320597,
 0.0001316936897183417,
 8,
 array([[3.42960289e-02, 4.47653430e-01, 1.51805054e+00],
        [1.72166065e+01, 2.24722022e+02, 7.62061372e+02],
        [1.61191336e+00, 2.10397112e+01, 7.13483755e+01],
        [8.57400722e-02, 1.11913357e+00, 3.79512635e+00],
        [5.14440433e-02, 6.71480144e-01, 2.27707581e+00]]))

In [16]:
two_way_table2 = pd.crosstab(index=df["BOROUGH"], columns=df["OFFENSE_NAME"])
two_way_table2

OFFENSE_NAME,ARSON,ASSAULT,BURGLARY,DRIVING_UNDER_INFLUENCE,FORGERY,FRAUD,GAMBLING,HARRASSMENT,HOMICIDE,KIDNAPPING,...,MURDER,OFFENSES_AGAINST_PUBLIC_ORDER,POSSESSION_CONTROLLED_SUBSSTANCE,POSSESSION_WEAPON,ROBBERY,SEX_CRIMES,SOCIAL_RELATED_CRIMES,THEFT,TRAFFIC_LAWS_VIOLATION,UNCLASSIFIED
BOROUGH,,,,,,,,,,,,,,,,,,,,,
BRONX,210,19904,2004,677,842,899,49,16754,2129,30,...,0,18548,4638,1585,3517,1483,428,572,1852,195
BROOKLYN,198,20421,3626,1388,1810,1555,157,20291,5403,38,...,0,22409,3149,2262,4016,2153,347,858,2586,266
MANHATTAN,117,15120,2577,688,1136,2049,76,14906,2091,22,...,0,16550,3807,1091,3089,1967,151,991,1355,266
QUEENS,119,15096,2456,969,800,1182,6,15171,3556,32,...,0,15576,1286,1091,2476,1688,287,538,2221,90
STATEN ISLAND,31,2559,284,644,94,354,1,4031,880,4,...,0,4064,487,251,227,244,54,93,237,82
UNKNOWN,0,2,0,0,1,0,0,1,0,0,...,292,0,0,0,0,0,0,1,0,0


In [17]:
two_way_table3 = pd.crosstab(index=df["VIC_AGE_GROUP"], columns=df["OFFENSE_NAME"])
two_way_table3

OFFENSE_NAME,ARSON,ASSAULT,BURGLARY,DRIVING_UNDER_INFLUENCE,FORGERY,FRAUD,GAMBLING,HARRASSMENT,HOMICIDE,KIDNAPPING,...,MURDER,OFFENSES_AGAINST_PUBLIC_ORDER,POSSESSION_CONTROLLED_SUBSSTANCE,POSSESSION_WEAPON,ROBBERY,SEX_CRIMES,SOCIAL_RELATED_CRIMES,THEFT,TRAFFIC_LAWS_VIOLATION,UNCLASSIFIED
VIC_AGE_GROUP,,,,,,,,,,,,,,,,,,,,,
18-24,8,12126,550,38,36,309,0,7614,1196,19,...,55,5732,8,137,2322,1285,182,76,873,3
25-44,72,34716,3403,151,133,1811,1,32378,5522,49,...,151,26127,70,416,4985,2218,507,545,3154,24
45-64,64,14743,1994,102,199,1343,0,19964,2527,8,...,43,14233,48,233,2195,497,203,542,1792,20
65+,21,2039,601,18,116,549,0,3948,620,1,...,20,3087,7,21,426,72,32,111,452,1
<18,0,6081,34,4,7,27,0,3858,376,37,...,16,1460,5,35,1674,3032,62,15,425,4
UNKNOWN,510,3397,4365,4053,4192,2000,288,3392,3818,12,...,7,26508,13229,5438,1723,431,281,1764,1555,847


In [18]:
# try multi-indexing for contingency table

result_chi2 = stats.chi2_contingency(observed = two_way_table2)


chi2, p, dof, expected = stats.chi2_contingency(observed = two_way_table2)

print('chi-square statistic :', result_chi2[0])
print('p-value :', result_chi2[1])
print('degrees of freedom :', result_chi2[2])
print('expected counts : \n', result_chi2[3])

table2 = sm.stats.Table(two_way_table2)
table2.standardized_resids

chi-square statistic : 458027.491792924
p-value : 0.0
degrees of freedom : 100
expected counts : 
 [[1.48180953e+02 1.60478875e+04 2.40316577e+03 9.58456359e+02
  1.02804653e+03 1.32572560e+03 6.34434008e+01 1.56202482e+04
  3.08633485e+03 2.76604446e+01 2.94434457e+04 6.41019828e+01
  1.69358756e+04 2.93442193e+03 1.37863168e+03 2.92520178e+03
  1.65413849e+03 2.78141138e+02 6.70216964e+02 1.81132007e+03
  1.97355077e+02]
 [1.93634168e+02 2.09704370e+04 3.14031591e+03 1.25245449e+03
  1.34339083e+03 1.73238036e+03 8.29041106e+01 2.04116231e+04
  4.03304115e+03 3.61450448e+01 3.84749658e+04 8.37647069e+01
  2.21308077e+04 3.83453027e+03 1.80151493e+03 3.82248192e+03
  2.16153105e+03 3.63458506e+02 8.75800172e+02 2.36692670e+03
  2.57892026e+02]
 [1.70105304e+02 1.84222785e+04 2.75873003e+03 1.10026631e+03
  1.18015280e+03 1.52187546e+03 7.28302711e+01 1.79313672e+04
  3.54297848e+03 3.17529902e+01 3.37997980e+04 7.35862947e+01
  1.94416503e+04 3.36858904e+03 1.58260935e+03 3.35800471e+

OFFENSE_NAME,ARSON,ASSAULT,BURGLARY,DRIVING_UNDER_INFLUENCE,FORGERY,FRAUD,GAMBLING,HARRASSMENT,HOMICIDE,KIDNAPPING,...,MURDER,OFFENSES_AGAINST_PUBLIC_ORDER,POSSESSION_CONTROLLED_SUBSSTANCE,POSSESSION_WEAPON,ROBBERY,SEX_CRIMES,SOCIAL_RELATED_CRIMES,THEFT,TRAFFIC_LAWS_VIOLATION,UNCLASSIFIED
BOROUGH,,,,,,,,,,,,,,,,,,,,,
BRONX,5.740645,37.644161,-9.332245,-10.343810,-6.601833,-13.358077,-2.066901,11.191437,-19.818080,0.479129,...,-9.033859,15.402989,36.135574,6.332706,12.571307,-4.805480,10.176413,-4.308508,1.089837,-0.198475
BROOKLYN,0.359761,-4.904890,10.388031,4.553259,15.155080,-5.083743,9.612895,-1.086042,25.951075,0.336623,...,-10.823165,2.434498,-13.309419,12.934125,3.761017,-0.221788,-1.032047,-0.714039,5.379128,0.588070
MANHATTAN,-4.720655,-30.729471,-4.051691,-14.443848,-1.493150,15.725550,0.412337,-28.461989,-28.656187,-2.023254,...,-9.896965,-26.335294,8.864851,-14.391357,-5.449856,1.820292,-10.910963,9.270312,-18.538404,3.022917
QUEENS,-1.542909,4.887230,6.477147,3.657498,-4.998372,-0.817258,-7.625070,9.684398,15.975881,1.492901,...,-8.507446,1.533336,-30.430354,-5.223800,-4.125532,5.279308,2.366105,-3.281294,15.881033,-7.494457
STATEN ISLAND,0.635535,-8.990134,-8.063282,35.626034,-7.271718,6.928974,-3.224863,22.866045,13.083357,-0.534325,...,-3.404593,17.889688,-2.728895,-0.434192,-14.181057,-3.823144,0.280199,-2.957334,-5.694315,7.583978
UNKNOWN,0.060455,-7.392687,-2.576684,-1.439885,-1.230282,-1.791819,0.684118,-7.423190,-2.977149,1.412734,...,652.678257,-7.879872,-2.892371,-1.837443,-2.887155,-2.060056,-0.390086,-0.749006,-2.177703,-0.142564
